In [18]:
import bs4
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

# from langchain_core.prompts import ChatPromptTemplate
# from langchain_ollama.llms import OllamaLLM
# from langchain.llms import CTransformers

#embeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

prompt = PromptTemplate (
    input_variables=["context", "question"],
    template="""
    You are an assistant for question-answering tasks. 
    Use documentation to answer the question.
    If you don't know the answer, just say you don't know.
    Be as concise as possible.
    {context}
    Question: {question}
    """
)

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

llm = ChatOllama (
      model="llama3.1:8b",
      base_url="http://127.0.0.1:11434",
      temperature=0.3
      )

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("What is Task Decomposition?"))
print("\n")
print(rag_chain.invoke("What do dogs like to eat?"))



c:\Users\Dexter\Documents\GitHub\sample-chatbot\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Task Decomposition is a technique that breaks down complicated tasks into smaller, more manageable steps. It involves instructing a model to "think step by step" (Chain of Thought) to decompose hard tasks into simpler ones.


```json
{
  "thoughts": {
    "text": "Dogs have specific dietary needs.",
    "reasoning": "Based on general knowledge about dog nutrition.",
    "plan": "- Research common dog foods.\n- Consider canine nutritional requirements.\n- Identify popular dog food types (e.g., kibble, raw).",
    "criticism": "More information about individual dog breeds' dietary needs would be helpful.",
    "speak": "To provide a comprehensive answer, I'd like to know what type of dogs you're referring to."
  },
  "command": {
    "name": "ask_dog_food_type",
    "args": {
      "dog_breed": "{{user input text}}"
    }
  }
}
```
